In [3]:
import pandas as pd
import os
import argparse
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
#from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Import the first CSV file
df1 = pd.read_csv('../Datasets/superconduct/train.csv')

# Import the second CSV file
df2 = pd.read_csv('../Datasets/superconduct/unique_m.csv')
df2 = df2.drop(['critical_temp', 'material'], axis=1)

combined_df = df1.join(df2)

In [ ]:
combined_df

In [4]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

In [5]:
# Create train/test split
# Split the data into X and y
X = combined_df.drop('critical_temp', axis=1)
y = combined_df['critical_temp']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

 # Add column names back into the scaled X_train data frame
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)

model = Lasso()
param_dist = {
    "alpha": np.arange(0, 1.01, 0.01)    #np.arange(0, 5.0, 0.01) for neg_absolute_error
}
grid_search = GridSearchCV(model, param_grid=param_dist, scoring='r2', cv=5) #neg_mean_absolute_error suggested 0.01
grid_search.fit(X_scaled, y)

print(grid_search.best_params_)
#report(grid_search.cv_results_)

/home/xclusive/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/xclusive/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/xclusive/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.941e+06, tolerance: 1.686e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Rid

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
vi = []
for i in np.arange(0, 1.01, 0.01):
    model.alpha = i
    model.fit(X_scaled,y)
    vi.append(model.coef_)

# Plot the coefficients
V=pd.DataFrame(np.array(vi),columns=X_scaled.columns)
for i in V.columns:
    plt.plot(np.arange(0, 1.01, 0.01),V[i],label=i)

plt.plot(np.arange(0, 1.01, 0.01), vi)
plt.xlabel('Iteration')
plt.ylabel('Coefficients')
plt.title('Convergence of Lasso Coefficients')
#plt.legend()
plt.show()
    
    
#V=pd.DataFrame(np.array(vi),columns=X_scaled.columns)
#import matplotlib.pyplot as plt
#for i in V.columns:
#    plt.plot(np.arange(0, 5.0, 0.01),V[i],label=i)
#plt.plot(np.ones(100)*.002,np.linspace(-1,1,100))
#plt.xscale('linear')
#plt.legend()
#plt.show()

In [6]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Add column names back into the scaled X_train data frame
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)

In [ ]:
# Instantiate a Lasso regressor
l1_model = Lasso(alpha=.35)               #r2 metric from gridsearch said .35 was the best alpha between 0-1. neg_mean_absolute_error

# Fit the regressor to the data
l1_model.fit(X_train, y_train)

# Predict on the test set
y_pred = l1_model.predict(X_test)

# Compute the MSE of the Lasso model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)

In [ ]:
non_zero_coef = []
for i in range(len(X_scaled.columns)):
    if l1_model.coef_[i] != 0:
        non_zero_coef.append((X_scaled.columns[i], abs(l1_model.coef_[i])))
print(sorted(non_zero_coef, key=lambda x:x[1], reverse = True))

print(len(non_zero_coef))

In [12]:
# Create train/test split
# Split the data into X and y
X = combined_df.drop('critical_temp', axis=1)
y = combined_df['critical_temp']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

 # Add column names back into the scaled X_train data frame
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)

model = Ridge()
param_dist = {
    'alpha': np.logspace(-6, 6, 10)
}
grid_search = GridSearchCV(model, param_grid=param_dist, scoring='neg_mean_squared_error', cv=5) #neg_mean_absolute_error suggested 0.01
grid_search.fit(X_scaled, y)

print("Best regularization strength: ", grid_search.best_params_)
print("Best mean squared error: ", -grid_search.best_score_)

ridge_alpha = grid_search.best_params_
#report(grid_search.cv_results_)

Best regularization strength:  {'alpha': 2976.3514416313133}
Best mean squared error:  376.0095670559369


In [ ]:
import matplotlib.pyplot as plt
vi = []
for i in np.logspace(-6, 6, 10):
    model.alpha = i
    model.fit(X_scaled,y)
    vi.append(model.coef_)

# Plot the coefficients
V=pd.DataFrame(np.array(vi),columns=X_scaled.columns)
for i in V.columns:
    plt.plot(np.logspace(-6, 6, 10),V[i],label=i)

plt.plot(np.logspace(-6, 6, 10), vi)
plt.xlabel('Iteration')
plt.ylabel('Coefficients')
plt.title('Convergence of Lasso Coefficients')
#plt.legend()
plt.show()

In [10]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Add column names back into the scaled X_train data frame
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)

In [11]:
# Instantiate a Lasso regressor
l2_model = Ridge(alpha=ridge_alpha['alpha'])               #r2 metric from gridsearch said .35 was the best alpha between 0-1. neg_mean_absolute_error

# Fit the regressor to the data
l2_model.fit(X_train, y_train)

# Predict on the test set
y_pred = l2_model.predict(X_test)

# Compute the MSE of the Lasso model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)

NameError: name 'ridge_alpha' is not defined

In [9]:
non_zero_coef = []
for i in range(len(X_scaled.columns)):
    if l2_model.coef_[i] != 0:
        non_zero_coef.append((X_scaled.columns[i], abs(l2_model.coef_[i])))
print(sorted(non_zero_coef, key=lambda x:x[1], reverse = True))

print(len(non_zero_coef))

[('wtd_entropy_Valence', 35.7126281920486), ('wtd_entropy_fie', 25.483936903563606), ('wtd_entropy_FusionHeat', 19.990256436286256), ('entropy_atomic_mass', 16.269318832668564), ('wtd_std_Valence', 15.930976186210946), ('entropy_Valence', 15.427375791278797), ('wtd_entropy_ElectronAffinity', 15.340129796486115), ('entropy_FusionHeat', 15.336386733809487), ('entropy_ThermalConductivity', 13.01622950100774), ('Ag', 11.450766314936418), ('F', 8.67201988053719), ('wtd_entropy_atomic_radius', 8.418845433272086), ('Ba', 7.660216626626173), ('Rb', 7.5141472893618415), ('K', 7.1945507806237625), ('entropy_ElectronAffinity', 6.990190974842229), ('Hg', 6.972053577814011), ('entropy_fie', 6.5581017653692095), ('wtd_entropy_atomic_mass', 6.339183258287006), ('Cl', 6.161781739351226), ('I', 6.0951449395458805), ('gmean_Valence', 6.027546475954805), ('Bi', 5.984227893744472), ('entropy_Density', 5.863701368641587), ('range_Valence', 5.499786923885246), ('Na', 5.1129113808222), ('mean_Valence', 5.071